In [13]:
import logging
import sys
import requests
from ct.aws import get_secret
from ct.utils import plt
from ct.utils import glassnode
from ct.utils import DF,npa,np,pd
import backtrader as bt
import ipdb
import time
from datetime import timedelta
from datetime import datetime as dt
from ccxtbt1 import CCXTStore
from ct.utils import qbars,date
from backtrader_plotting.schemes import Blackly
from backtrader_plotting.schemes import Tradimo
from backtrader_plotting.bokeh.live.plotlistener import PlotListener
from ct.alfafactory import trade_list,WalletObserver,SumWalletObserver,CloseObserver,PositionObserver,printTradeAnalysis
from ct.fs import addbb2

In [14]:
PRINTCANDLE=True
class GlassnodeStrategy(bt.Strategy):
    params = (('sym','BTC'),('betsize',0.001),('lags', 20),('delay',4),('trailpercent', 3),('inflowlimit',5),('inflowlimit2',12),('nstd',2))# ('entryz', 1.2), ('exitz',1.2),('methodma','ewm'),('methodstd','ewm'),('trailpercent', 2),('liqlimit',35))
    
    def notify_order(self, order): 
        if not order.alive():
            self.order = None  # No pending orders
        logging.critical(f"|notify_order|{order}|position={self.position.size}|brokerpos={self.broker.getposition(self.datas[0]).size}|c={self.data0.close[0]}|o={self.data0.open[0]}")
    def __init__(self):
        print("strat init()")
        self.order = None
        
    def start(self):
        print("strat start()")
        self.order = None  # Avoid operrations on pending order

    def next(self):
        if not BACKTEST:
            if self.live_data==False:
                return
        if self.order:
            return  # pending order execution

        curdt=bt.num2date(self.data0.datetime[0])
        o,h,l,c,v=self.data0.open[0],self.data0.high[0],self.data0.low[0],self.data0.close[0],self.data0.volume[0]
        if PRINTCANDLE:print(f"curdt={curdt} c={c}")
        
        if BACKTEST:
            dfpn=dfgsi#[::-1]
            dfp=dfpn[dfpn.index<curdt].head(self.p.lags)['v']
        else:
            dfp=glassnode('https://api.glassnode.com/v1/metrics/transactions/transfers_volume_to_exchanges_mean',a=self.p.sym,i='10m',s=dt.now()-timedelta(hours=2),Nretry=10)['v']
        if PRINTCANDLE:print(f"\n {dt.now()} next() self.dt={self.datetime.datetime(0)} data0close={self.data0.close[0]} data1close={self.data1.close[0] if hasattr(self,'data1') else 'not present'}  data1close={self.data2.close[0] if hasattr(self,'data2') else 'not present'}  \n")    
        if PRINTCANDLE:print(f"data0 get5={self.data0.close.get(size=5) if hasattr(self,'data0') else 'not present'}")

        if self.position.size==0: # not in the market
            if PRINTCANDLE:print(f"dfp.iloc[0] {dfp.iloc[0]}")
            if BACKTEST:
                condition=(len(dfp)>5 and dfp.iloc[self.p.delay]>self.p.inflowlimit and dfp.iloc[self.p.delay]<self.p.inflowlimit2)
            else:
                logging.error(f'\n{dfp}')
                condition=(dfp.iloc[-1]>self.p.inflowlimit and dfp.iloc[-1]<self.p.inflowlimit2)
            print('condition={}'.format(condition))        
            if condition :
                self.order = self.sell(data=self.data0,exectype=bt.Order.Market,size=self.p.betsize)
                
                logging.critical(f"{self.order}")
                self.order = None

        elif self.order is None: # Position in Market
            self.order = self.buy(data=self.data0,exectype=bt.Order.StopTrail,trailpercent=self.p.trailpercent/100,price=self.data0.open[0],size=self.p.betsize,params={'reduceOnly':True,'callbackRate':self.p.trailpercent}) #If no price is specified, the latest close price is used
            logging.critical(f"{self.order}")

    def notify_data(self, data, status, *args, **kwargs):
        if data._getstatusname(status) == 'LIVE':
            self.live_data = True
        else:
            self.live_data = False

    def log(self,txt,dummy=True):
        logging.debug(txt)

In [15]:
API_KEY = '1wvlrc0JsEMIWF6enI69cWiOojq'

def glassnode(url='https://api.glassnode.com/v1/metrics/transactions/transfers_volume_to_exchanges_mean',a='BTC',i='10m',s=1629500000):
    res = requests.get(url,params={'a': a,'i':i,s:s,'api_key': API_KEY}) #, 's':1629500000
    df = pd.read_json(res.text, convert_dates=['t']).set_index('t') 
    return df
                       
dfgsi=glassnode('https://api.glassnode.com/v1/metrics/transactions/transfers_volume_to_exchanges_mean',a='BTC',i='10m')[::-1]

In [16]:
display(dfgsi.head())

,v
t,
2021-11-10 09:50:00,0.4672896903
2021-11-10 09:40:00,0
2021-11-10 09:30:00,0.07970281445
2021-11-10 09:20:00,0.2357675184
2021-11-10 09:10:00,0.2808873212


In [17]:
BOTNAME='GLASSNODEMINIMAL'

LOGFILENAME=f'/core/logs/{BOTNAME}.log'#'/core/logs/livetestminimal.log'

logging.basicConfig(level=logging.ERROR,filename=LOGFILENAME,format="{asctime}| [{levelname:4}]|{message}|",style="{",force=True)

root = logging.getLogger()
#root.setLevel(logging.ERROR)
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.ERROR)
root.addHandler(handler)
#handler.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))

MONITORPORT=50005

COIN_TARGET = "ETH"
COIN_REFER = "USDT"

DEBUG = True
BACKTEST=True

LAGS=3
MINUTES=5





def main():
    cerebro = bt.Cerebro(stdstats=False) #quicknotify=True)
    cerebro.addstrategy(GlassnodeStrategy,sym=COIN_TARGET,betsize=0.002,lags=20,delay=4,trailpercent=3,inflowlimit=5,inflowlimit2=12)
    cerebro.addobserver(bt.observers.Trades)
    cerebro.addobserver(bt.observers.BuySell,barplot=True,bardist=0.02)
    cerebro.addanalyzer(bt.analyzers.TimeReturn)#,_name='timereturn')
    
    if BACKTEST is False: 
        cerebro.addobserver(WalletObserver,ccy='USDT')
        cerebro.addobserver(PositionObserver,ccys=['USDT'])
        cerebro.addlistener(bt.listeners.RecorderListener)
        cerebro.addlistener(PlotListener, volume=False, scheme=Tradimo(hovertool_timeformat='%F %R:%S', tabs='single'), lookback=480,http_port=MONITORPORT,address='0.0.0.0',debug=True)
        srt=get_secret('b1-wzxNSw')
        broker_config = { 'apiKey': srt['k'],'secret': srt['s'], 'nonce': lambda: str(int(time.time() * 1000)), 'enableRateLimit': True,'options': {'defaultType': 'future'}, }
        store = CCXTStore(exchange='binance', currency=COIN_REFER, config=broker_config, retries=5, debug=False)
        broker_mapping ={ 'order_types': { bt.Order.Market: 'market', bt.Order.Limit: 'limit', bt.Order.Stop: 'stop-loss', bt.Order.StopLimit: 'STOP_LOSS_LIMIT',
                                          bt.Order.StopTrail:'TRAILING_STOP_MARKET'},#bt.Order.StopTrailLimit:''}, 
         'mappings': {'closed_order': {'key': 'status','value': 'closed'},'canceled_order': {'key': 'status','value': 'canceled' } } }
        hist_start_date = dt.utcnow() - timedelta(minutes=LAGS*MINUTES)
        data = store.getdata( dataname=f'{COIN_TARGET}/{COIN_REFER}' , name=f'{COIN_TARGET}/{COIN_REFER}', timeframe=bt.TimeFrame.Minutes,fromdate=hist_start_date, compression=MINUTES, ohlcv_limit=LAGS*MINUTES, drop_newest=True )
        cerebro.adddata(data)#,timeframe=bt.TimeFrame.Minutes,compression=MINUTES)
        broker = store.getbroker(broker_mapping=broker_mapping)
        print('pos:',broker.getposition(data).size)
        print('possize',broker.positions[COIN_TARGET+'USDT'].size)
        print('balance:',broker.get_balance())
        print('open orders:',broker.private_end_point(type='GET',endpoint='/openOrders',params={'symbol':"BTCUSDT"}))
        print(broker.private_end_point(type='GET',endpoint='/openOrders',params={'symbol':"BTCUSDT"}))
        cerebro.setbroker(broker)
    elif BACKTEST:
        cerebro.addobserver(bt.observers.Value)
        cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta")
        cerebro.addanalyzer(trade_list,_name='tradelist')
        cerebro.addanalyzer(bt.analyzers.Transactions)
        dfbtc=qbars(COIN_TARGET+'USDT@binance',freq=str(MINUTES)+"Min",table='ppttrades',date=[date(2021,7,29),date(2021,8,4)],addwhere=', vol>0.01')
        dfbtc1=dfbtc#.loc['2021-05']
        data=bt.feeds.PandasData(dataname=dfbtc1.rename(columns={'o':'open','h':'high','l':'low','c':'close','v':'volume'}),openinterest=None,timeframe=bt.TimeFrame.Minutes,compression=MINUTES)
        cerebro.adddata(data)
        
    logging.critical("alfabot cerebro.run()")
    backtest = cerebro.run(runonce=False) if not BACKTEST else cerebro.run(runonce=False,tradehistory=True)
    final_value = cerebro.broker.getvalue()
    print('Final Portfolio Value: %.2f' % final_value)
    if BACKTEST:
        rcsave=plt.rcParams['figure.figsize']
        plt.rcParams['figure.figsize']=[30, 16]
        cerebro.plot(style='line', rowsmajor=1, rowsminor=1,volume=True, iplot=False, barup='green', bardown='grey',fmt_x_ticks = '%Y-%b-%d %H:%M',fmt_x_data = '%Y-%b-%d %H:%M')
        plt.rcParams['figure.figsize']=rcsave
        
        btres = backtest[0]
        for k in ['total','pnl']:#backtest[0].analyzers.ta.get_analysis():
            print (k,dict(backtest[0].analyzers.ta.get_analysis()[k]))
        printTradeAnalysis(cerebro, backtest[0].analyzers)
        print("amount,price,sid,symbol,value")
        display(DF(btres.analyzers.tradelist.get_analysis()).set_index('datein'))
        
try:
    main()
except KeyboardInterrupt:
    print("finished.")

query=system "l /core/data/db/binance";
0!select  o:first px,h:max px,l:min px,c:last px ,v:sum vol by date, symbol, 0D00:05 xbar exchtm from ppttrades where  date within 2021.07.29 2021.08.04,  symbol=`ETHUSDT , vol>0.01


FileNotFoundError: [WinError 2] The system cannot find the file specified